# Data cleaning

The current project 

## 1. Create a unique dataframe to work with

Because the chosen dataset contains 3 tables in three different documents is necessary to stack them into a single one. The three tables contains the same headers with the same type of data so, a more "complex" method to stack them is not necessary.

### 1.1 Inspect the data to assure they have the same columns headers

In [26]:
import pandas as pd

# Load the three CSV tables

kor_drama_path = r"C:\Users\lonel\Downloads\kor_drama.csv"
kor_drama = pd.read_csv(kor_drama_path)

jap_drama_path = r"C:\Users\lonel\Downloads\jap_drama.csv"
jap_drama = pd.read_csv(jap_drama_path)

thai_drama_path = r"C:\Users\lonel\Downloads\tha_drama.csv"
thai_drama = pd.read_csv(thai_drama_path)

## Look at the data

dramas_dic = {'kor_drama': kor_drama, 'jap_drama':jap_drama, 'thai_drama':thai_drama}

for drama_name, drama_data in dramas_dic.items():
    print(f"{drama_name} columns: ", drama_data.columns)
    print(f"{drama_name} info: ")
    print(drama_data.info())

kor_drama columns:  Index(['drama_id', 'drama_name', 'native_name', 'year', 'synopsis', 'genres',
       'tags', 'director', 'sc_writer', 'country', 'type', 'tot_eps',
       'ep_duration', 'start_dt', 'end_dt', 'aired_on', 'org_net',
       'tot_user_score', 'tot_num_user', 'tot_watched', 'content_rt', 'rank',
       'popularity'],
      dtype='object')
kor_drama info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   drama_id        1773 non-null   object 
 1   drama_name      1773 non-null   object 
 2   native_name     1766 non-null   object 
 3   year            1773 non-null   int64  
 4   synopsis        1604 non-null   object 
 5   genres          1768 non-null   object 
 6   tags            1773 non-null   object 
 7   director        1045 non-null   object 
 8   sc_writer       967 non-null    object 
 9   country         177

Based on the results, we can confirm that the data can be stack into a single dataframe and that some columns need to be cleaned.

### 1.2 Stack the data and inspect it

In [27]:
# Stack the data

## The keep traceability of where the data came from, let's add a column with an identifier

kor_drama['dataset'] = 'kor_drama'
jap_drama['dataset'] = 'jap_drama'
thai_drama['dataset'] = 'thai_drama'

## Stack the data

dramas = pd.concat(dramas_dic.values(), ignore_index=True)
print(f"dramas columns: ", dramas.columns)
print(f"dramas info: ")
print(dramas.info())

dramas columns:  Index(['drama_id', 'drama_name', 'native_name', 'year', 'synopsis', 'genres',
       'tags', 'director', 'sc_writer', 'country', 'type', 'tot_eps',
       'ep_duration', 'start_dt', 'end_dt', 'aired_on', 'org_net',
       'tot_user_score', 'tot_num_user', 'tot_watched', 'content_rt', 'rank',
       'popularity', 'dataset'],
      dtype='object')
dramas info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5442 entries, 0 to 5441
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   drama_id        5441 non-null   object 
 1   drama_name      5431 non-null   object 
 2   native_name     5418 non-null   object 
 3   year            5430 non-null   object 
 4   synopsis        4981 non-null   object 
 5   genres          5406 non-null   object 
 6   tags            5428 non-null   object 
 7   director        3511 non-null   object 
 8   sc_writer       3265 non-null   object 
 9   country       

## 2. Clean and transform the dataset

Because the Classification, Clustering and Association will use different atributes an specific dataframe for each task will be created and cleaned, to match the type of data each algorithm expects as an input

### 2.1 Create and clean an individual dataframes for Classification task

#### 2.1.1 Create the individual "Classification" dataframe

The Classification task requieres the columns popularity (as the target), genres, tot_eps, ep_duration, content_rt and country.

In [28]:
# Crate the dataframe

classification = dramas.copy()

# Keep only the needed columns

classification_cols = ['drama_name', 'popularity', 'genres', 'tot_eps', 'ep_duration',
                       'content_rt', 'country', 'dataset'] # Rember 'dataset' was created for traceability
                                                           # In addition, drama_name will also kept in the dataframe for traceability

classification = classification[classification_cols]

print("classification cols: ", classification.columns)
print("classification info: ")
print(classification.info())

classification cols:  Index(['drama_name', 'popularity', 'genres', 'tot_eps', 'ep_duration',
       'content_rt', 'country', 'dataset'],
      dtype='object')
classification info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5442 entries, 0 to 5441
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   drama_name   5431 non-null   object 
 1   popularity   5425 non-null   float64
 2   genres       5406 non-null   object 
 3   tot_eps      5399 non-null   object 
 4   ep_duration  5262 non-null   object 
 5   content_rt   5425 non-null   object 
 6   country      5428 non-null   object 
 7   dataset      5442 non-null   object 
dtypes: float64(1), object(7)
memory usage: 340.3+ KB
None


#### 2.1.2 Drop null values

Because we can find missing values in practically all the columns (except for "dataset" that was the column created), it seems to be a case of Missing at Random (MAR) and due to the low porcentage of missing data (about a 0.8% based on the missing data in tot_eps column), the chosen decision is to delete those rows with missing data, avoiding bias results in our Classification model.

The only null values that will not be droped will be those on the column "country" because they can be filled with the values on "dataset".

In [29]:
# Drop Nulls

null_cols = ['drama_name', 'popularity', 'genres', 'tot_eps', 'ep_duration', 'content_rt']

classification.dropna(subset=null_cols, inplace=True)

print("classification info:")
print(classification.info())

classification info:
<class 'pandas.core.frame.DataFrame'>
Index: 5208 entries, 0 to 5441
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   drama_name   5208 non-null   object 
 1   popularity   5208 non-null   float64
 2   genres       5208 non-null   object 
 3   tot_eps      5208 non-null   object 
 4   ep_duration  5208 non-null   object 
 5   content_rt   5208 non-null   object 
 6   country      5208 non-null   object 
 7   dataset      5208 non-null   object 
dtypes: float64(1), object(7)
memory usage: 366.2+ KB
None


Less than a 5% of the data was droped, so we still have enough data to work with.

#### 2.1.3 Transform the numerical data

Based on the info of the dataframe, the datatype of tot_eps and ep_duration is "object" when it should be int64 and float64 respectibly. Let's adjust the datatype.

In [30]:
# tot_eps

classification['tot_eps'] = pd.to_numeric(classification['tot_eps'], errors='coerce').astype(int)

# ep_duration

classification['ep_duration'] = pd.to_numeric(classification['ep_duration'], errors='coerce').astype(float)

# Valite there are not null values

print(classification.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5208 entries, 0 to 5441
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   drama_name   5208 non-null   object 
 1   popularity   5208 non-null   float64
 2   genres       5208 non-null   object 
 3   tot_eps      5208 non-null   int64  
 4   ep_duration  5208 non-null   float64
 5   content_rt   5208 non-null   object 
 6   country      5208 non-null   object 
 7   dataset      5208 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 366.2+ KB
None


#### 2.1.4 Transform the categorical data

The columns genres, content_rt and country are stores as object, but we need to trasnform the data into a format that Weka's Logistic Regression Model could work with.

#### 2.1.4.1 Transform genres column

In [31]:
# Perform EDA on the columns

print("genres unique", classification['genres'].nunique())
print("content_rt unique", classification['content_rt'].nunique())
print("country unique", classification['country'].nunique())

print("Classification data:")
print(classification[['genres', 'content_rt', 'country']].head())

genres unique 1098
content_rt unique 6
country unique 3
Classification data:
                                     genres  \
0       Thriller,  Mystery,  Comedy,  Drama   
1                                    Sitcom   
2  Historical,  Romance,  Drama,  Melodrama   
3          Music,  Comedy,  Romance,  Youth   
4  Action,  Mystery,  Comedy,  Supernatural   

                              content_rt      country  
0  18+ Restricted (violence & profanity)  South Korea  
1                          Not Yet Rated  South Korea  
2                15+ - Teens 15 or older  South Korea  
3                          Not Yet Rated  South Korea  
4                15+ - Teens 15 or older  South Korea  


Because genres is the only column that doesn't follows a "patter" it must be transformed into binary features.

In order to analyze the data and transform it, we should break it into small pieces

In [32]:
# Split the data

classification['genres'] = classification['genres'].fillna('').str.split(',')

print("classification['genres'] split: ", classification['genres'].head())

# Use the method explode to break each list into single data

classification_exploded = classification.explode('genres')
classification_exploded['genres'] = classification_exploded['genres'].str.strip()

print('Unique genres:', classification_exploded['genres'].unique())

classification['genres'] split:  0         [Thriller,   Mystery,   Comedy,   Drama]
1                                         [Sitcom]
2    [Historical,   Romance,   Drama,   Melodrama]
3            [Music,   Comedy,   Romance,   Youth]
4    [Action,   Mystery,   Comedy,   Supernatural]
Name: genres, dtype: object
Unique genres: ['Thriller' 'Mystery' 'Comedy' 'Drama' 'Sitcom' 'Historical' 'Romance'
 'Melodrama' 'Music' 'Youth' 'Action' 'Supernatural' 'Military' 'Fantasy'
 'Horror' 'Psychological' 'Life' 'Crime' 'Food' 'Adventure' 'Sci-Fi'
 'Business' 'Medical' 'Family' 'Political' 'Law' 'Mature' 'Sports'
 'Documentary' 'Martial Arts' 'Tokusatsu' 'War' 'Wuxia']


Once we know the true unique values and that are less than 50, we can use the MultiLabelBinarizer object from the Scikit-Learn library to transform it into binary features.

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb=MultiLabelBinarizer()

print(type(classification['genres'].iloc[0]))
print(classification['genres'].iloc[0])

# Clean the genres data
classification['genres'] = classification['genres'].apply(lambda row_list: [str.strip(item) for item in row_list])

# Create the matrix to tranform the genders into numeric values
genre_matrix = mlb.fit_transform(classification['genres'])

# Create a dataframe that contains the columns of the matrix
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=classification.index)
print("genre_df:")
print(genre_df.head())
print("genre_df: ", genre_df.columns)

# Join the matrix with the original dataframe
classification = classification.join(genre_df)

print("classification df:")
print(classification)
print("classification cols: ", classification.columns)

# Due to the transformation, the original genres column can be deleted

classification.drop(columns=['genres'], inplace=True)

<class 'list'>
['Thriller', '  Mystery', '  Comedy', '  Drama']
genre_df:
   Action  Adventure  Business  Comedy  Crime  Documentary  Drama  Family  \
0       0          0         0       1      0            0      1       0   
1       0          0         0       0      0            0      0       0   
2       0          0         0       0      0            0      1       0   
3       0          0         0       1      0            0      0       0   
4       1          0         0       1      0            0      0       0   

   Fantasy  Food  ...  Romance  Sci-Fi  Sitcom  Sports  Supernatural  \
0        0     0  ...        0       0       0       0             0   
1        0     0  ...        0       0       1       0             0   
2        0     0  ...        1       0       0       0             0   
3        0     0  ...        1       0       0       0             0   
4        0     0  ...        0       0       0       0             1   

   Thriller  Tokusatsu  War  W

#### 2.1.4.2 Transform country column

As we saw at the beggining of this point (2.1.4), country contains three strings (South Korea, Japan and Thailand), so to convert the data we will use pandas get_dummies method.

In [ ]:
# Create dummies
countries = pd.get_dummies(classification['country'])

# Join them with original dataframe

classification = classification.join(countries)

print("classification: ")
print(classification.head())
print("classification columns: ", classification.columns)

# Drop country column
classification.drop(columns=['country'], inplace=True)

classification: 
                                    drama_name  popularity  tot_eps  \
0                                    Mask Girl      1052.0        7   
1                                Better Things     99999.0        3   
2                                   My Dearest      1584.0       10   
3                                Sing My Crush      1628.0        8   
4  The Uncanny Counter Season 2: Counter Punch       420.0       12   

   ep_duration                             content_rt      country    dataset  \
0       3600.0  18+ Restricted (violence & profanity)  South Korea  kor_drama   
1        960.0                          Not Yet Rated  South Korea  kor_drama   
2       4800.0                15+ - Teens 15 or older  South Korea  kor_drama   
3       1500.0                          Not Yet Rated  South Korea  kor_drama   
4       4200.0                15+ - Teens 15 or older  South Korea  kor_drama   

   Action  Adventure  Business  ...  Sports  Supernatural  Thriller  

#### 2.1.4.2 Transform content_rt column

To transform content_rt column we will use the OrdinalEncoder sklrn's function.

In [ ]:
# Find the unique values
print("content_rt unique values")
print(classification['content_rt'].unique().tolist())


# Encode the data
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[['Not Yet Rated', 'G - All Ages', '13+ - Teens 13 or older',
                                      '15+ - Teens 15 or older', '18+ Restricted (violence & profanity)',
                                      'R - Restricted Screening (nudity & violence)']])

# Add the data to the original df

classification['content_rt_encoded'] = encoder.fit_transform(classification[['content_rt']])

print(classification[['content_rt', 'content_rt_encoded']].head())

classification.drop(columns=['content_rt'], inplace=True)

content_rt unique values
['18+ Restricted (violence & profanity)', 'Not Yet Rated', '15+ - Teens 15 or older', '13+ - Teens 13 or older', 'G - All Ages', 'R - Restricted Screening (nudity & violence)']
                              content_rt  content_rt_encoded
0  18+ Restricted (violence & profanity)                 4.0
1                          Not Yet Rated                 0.0
2                15+ - Teens 15 or older                 3.0
3                          Not Yet Rated                 0.0
4                15+ - Teens 15 or older                 3.0


####  Save the dataframe